### LSTM结构

In [1]:
from torch import nn

# Hyper Parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 64
TIME_STEP = 100          # rnn时间点数量
INPUT_SIZE = 2           # rnn每个时间点输入的特征量 流速、温度
LR = 0.01                # learning rate

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=INPUT_SIZE,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 1)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out


rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(2, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [2]:
from torch import nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 408, 1024)
            nn.Conv2d(
                in_channels=1,  # input height
                out_channels=16,  # n_filters
                kernel_size=5,  # filter size
                stride=1,  # filter movement/step
                padding=2,
                # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),  # output shape (16, 408, 1024)
            nn.ReLU(),  # activation
            nn.MaxPool2d(kernel_size=2),  # choose max value in 2x2 area, output shape (16, 204, 512)
        )
        self.conv2 = nn.Sequential(  # input shape (16, 204, 512)
            nn.Conv2d(16, 32, 5, 1, 2),  # output shape (32, 204, 512)
            nn.ReLU(),  # activation
            nn.MaxPool2d(2),  # output shape (32, 102, 256)
        )
        self.conv3 = nn.Sequential(  # input shape (32, 102, 256)
            nn.Conv2d(32, 32, 5, 1, 2),  # output shape (32, 102, 256)
            nn.ReLU(),  # activation
            nn.MaxPool2d(2),  # output shape (32, 51, 128)
        )
        self.conv4 = nn.Sequential(  # input shape (32, 51, 128)
            nn.Conv2d(32, 32, 5, 1, 2),  # output shape (32, 51, 128)
            nn.ReLU(),  # activation
            nn.MaxPool2d(2),  # output shape (32, 25, 64)
        )
        self.classification = nn.Linear(32 * 25 * 64, 40)  # fully connected layer, output 20 classes
        self.out = nn.Linear(40, 2)  # fully connected layer, output 2 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)  # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = self.classification(x)
        output = self.out(x)
        return output  # return x for visualization

cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification): Linear(in_features=51200, out_features=40, bias=True)
  (out): Linear(in_features=40, out_features=2, bias=True)
)
